In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn.conv import GCNConv, SAGEConv, GINConv, GATConv
from torch_geometric.datasets import Planetoid
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
data_dir = "~/tmp/data"
cora = Planetoid(root=data_dir, name="Cora")

print(f"Number of classes: {cora.num_classes}")
print(f"Number of features: {cora.num_features}")
print(f"Number of nodes: {cora.data.num_nodes}")
print(f"Number of edges: {cora.data.num_edges}")
print(f"Training nodes: {cora.data.train_mask.sum()}")
print(f"Val nodes: {cora.data.val_mask.sum()}")
print(f"Test nodes: {cora.data.test_mask.sum()}")

Number of classes: 7
Number of features: 1433
Number of nodes: 2708
Number of edges: 10556
Training nodes: 140
Val nodes: 500
Test nodes: 1000


In [3]:
print(cora.data.y[0])
print(cora.data.x.shape)
print(cora.data.train_mask)

tensor(3)
torch.Size([2708, 1433])
tensor([ True,  True,  True,  ..., False, False, False])


In [4]:
class GCN(nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(cora.num_features, 128)
        self.conv2 = GCNConv(128, 32)
        self.conv3 = GCNConv(32, cora.num_classes)
                
    def forward(self, h, edge_index):
        h = self.conv1(h, edge_index)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.conv2(h, edge_index)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.conv3(h, edge_index)
        
        return h

In [5]:
class SAGE(nn.Module):
    def __init__(self):
        super(SAGE, self).__init__()
        torch.manual_seed(12345)
        self.layer1 = SAGEConv(cora.num_features, 128)
        self.layer2 = SAGEConv(128, cora.num_classes)
        
    def forward(self, h, edge_index):
        h = self.layer1(h, edge_index)
        h = h.relu()
        #h = F.dropout(h, p=0.5, training=self.training)
        h = self.layer2(h, edge_index)
        
        return h        

In [6]:
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.layer1 = GATConv(cora.num_features, 8, heads=8, dropout=0.6)
        self.layer2 = GATConv(64, cora.num_classes, heads=1, dropout=0.6)
    
    def forward(self, h, edge_index):
        h = self.layer1(h, edge_index)
        h = F.elu(h)
        h = self.layer2(h, edge_index)
        
        return F.softmax(h, dim=1)

In [25]:
def train(model, data, epochs, epoch_log, criterion, optimizer, patience, monitor):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            model.eval()
            pred = out.argmax(dim=1)
            correct_train = (pred[data.train_mask] == data.y[data.train_mask]).sum()
            correct_val = (pred[data.val_mask] == data.y[data.val_mask]).sum()
            train_acc = correct_train / data.train_mask.sum()
            val_acc = correct_val / data.val_mask.sum()
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                    
        if epoch % epoch_log == 0:
            print(f"epoch {epoch} - train_loss: {loss} - train_acc: {train_acc:0.4f} - val loss: {val_loss} - val_acc: {val_acc:0.4f}")
        
        if 
        
def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    results = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = results / data.test_mask.sum()
    print(acc)

In [26]:
epochs = 1000
epoch_log = 10
weight_decay = 5e-4
lr = 0.005
criterion = nn.CrossEntropyLoss()

model = GAT().to(device)
data = cora.data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
print(model)

GAT(
  (layer1): GATConv(1433, 8, heads=8)
  (layer2): GATConv(64, 7, heads=1)
)


In [27]:
print(f"Using {device} to train.")
train(model, data, epochs, epoch_log, criterion, optimizer)

Using cuda:0 to train.
epoch 0 - train_loss: 1.9419984817504883 - train_acc: 0.1643 - val loss: 1.944545030593872 - val_acc: 0.1440
epoch 10 - train_loss: 1.5857954025268555 - train_acc: 0.7643 - val loss: 1.7023471593856812 - val_acc: 0.6580
epoch 20 - train_loss: 1.4634510278701782 - train_acc: 0.7929 - val loss: 1.5808606147766113 - val_acc: 0.6820
epoch 30 - train_loss: 1.3651468753814697 - train_acc: 0.8357 - val loss: 1.5595567226409912 - val_acc: 0.6640
epoch 40 - train_loss: 1.3078250885009766 - train_acc: 0.9071 - val loss: 1.5460660457611084 - val_acc: 0.6740
epoch 50 - train_loss: 1.3330531120300293 - train_acc: 0.8643 - val loss: 1.5289453268051147 - val_acc: 0.6640
epoch 60 - train_loss: 1.3466776609420776 - train_acc: 0.8500 - val loss: 1.5238271951675415 - val_acc: 0.6660
epoch 70 - train_loss: 1.3376109600067139 - train_acc: 0.8714 - val loss: 1.545123815536499 - val_acc: 0.6560
epoch 80 - train_loss: 1.3335446119308472 - train_acc: 0.8571 - val loss: 1.532637596130371 

epoch 760 - train_loss: 1.303703784942627 - train_acc: 0.8714 - val loss: 1.522100806236267 - val_acc: 0.6520
epoch 770 - train_loss: 1.3239892721176147 - train_acc: 0.8500 - val loss: 1.5233862400054932 - val_acc: 0.6340
epoch 780 - train_loss: 1.265628457069397 - train_acc: 0.9071 - val loss: 1.519695520401001 - val_acc: 0.6380
epoch 790 - train_loss: 1.290931224822998 - train_acc: 0.9000 - val loss: 1.5371614694595337 - val_acc: 0.6340
epoch 800 - train_loss: 1.2920118570327759 - train_acc: 0.9071 - val loss: 1.518775224685669 - val_acc: 0.6540
epoch 810 - train_loss: 1.2717015743255615 - train_acc: 0.8929 - val loss: 1.5351539850234985 - val_acc: 0.6300
epoch 820 - train_loss: 1.2683625221252441 - train_acc: 0.9143 - val loss: 1.5338022708892822 - val_acc: 0.6100
epoch 830 - train_loss: 1.2758690118789673 - train_acc: 0.8929 - val loss: 1.5189663171768188 - val_acc: 0.6400
epoch 840 - train_loss: 1.3064175844192505 - train_acc: 0.8857 - val loss: 1.5215811729431152 - val_acc: 0.632

In [11]:
test(model, data)

tensor(0.8170, device='cuda:0')
